---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---

---
# Answer 3
Here are my thoughts on the approach:
* Use a multi-layer LSTM encoder - decoder as explained in the translate example in Tensorflow
* ---Build the batches so that input words are given in reverse order but not mirrored.  For example, "Now is the time for all good people" would be input into the encoder as "people good all for time the is Now".  This will help the LSTM to remember things closer to where the decoder will need them (just as it helped in translating English to French).--- (EDIT maybe do this in a v2, but start with simple character encoding)
* Also build the batches such that the target labels are the mirrored words, i.e. "won si eht emit rof lla doog elpoep".
* Keep it simple by using a character model and using one-hot encoding since the vocabulary size will be small for a character-based model.  That means I don't think I'll need embeddings for inputs or output_projection for outputs (which is the equivalent of embeddings for inputs which prevent you from having to specify a one-hot for a large vocab on the output -- it is used differently than embeddings though -- you do a matrix multiply + bias to get the one-hot logits and then use sample_softmax to get the predicted percentages.)
* Use the end-of-sentence token and pad to a certain length (you could use buckets to make it more efficient for padding, but let's keep it simple for now."
* Make it a character model, not a word model.
---

In [3]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    if not os.path.exists(filename):
        filename, _ = urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified %s' % filename)
    else:
        print(statinfo.st_size)
        raise Exception(
          'Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename

filename = maybe_download('text8.zip', 31344016)

def read_data(filename):
    f = zipfile.ZipFile(filename)
    for name in f.namelist():
        return tf.compat.as_str(f.read(name))
    f.close()

text = read_data(filename)
print('Data size %d' % len(text))

# Create small validation set

valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print('training data size: %s %s' % (train_size, train_text[:64]))
print('validation data size: %s %s' % (valid_size, valid_text[:64]))


Found and verified text8.zip
Data size 100000000
training data size: 99999000 ons anarchists advocate social relations based upon voluntary as
validation data size: 1000  anarchism originated as a term of abuse first used against earl


In [4]:
import collections, re
# DAY
#
# This is important to understand.  Our NN needs a constant sized vector with each input.  We are
# providing that here.  As the video says, just as convolution lets us use the same weight parameters
# at different parts of the image, a recurrent neural net lets us use the same weights at different
# points in time (or rather, different points in the input sequence).
#
# The notion of "unrollings" is that a recurrent NN has it's output connected to it's input, but really
# the way to think about it is over time where the output of time t-1 is input to time t.  That way
# of looking at it is like "unrolling" the recurrent NN over time so it is understood more as a
# sequence of copies of the NN.  
# In this case, we are going to be feeding in sequences that are 10 long, so we will in effect
# create 10 LSTM cells (which are really just a NN) and hook the output of LSTM cell t with inputs
# from input_sub_t and also the output of LSTM cell t-1.

# I'm re-writing the BatchGenerator to be more general purpose.  I want it to always output
# IDs (not embeddings, not the actual text, but the ID of the embeddings)

class SequenceGenerator(object):
    '''This class will take a text input and generate batches of IDs suitable for use with
    tf.nn.embedding_lookup() (i.e. goes from 0 to len(vocab)-1).  This class can be 
    inherited to create classes that create IDs for single characters, bigrams, 
    or entire words.  It also has the ability to take
    ID output from the RNN and convert it back to the original text.'''
    def __init__(self, text, batch_size, num_unrollings, vocab_size_limit=None):
        self._text = text
        self._batch_size = batch_size
        self._num_unrollings = num_unrollings
        self._vocab_size_limit = vocab_size_limit
        self._init_vocab()
        self._init_token_sequence()
        self._text = None # garbage collect now that self._token_seq is written
        segment = self._token_seq_size // batch_size
        self._cursor = [ offset * segment for offset in range(batch_size)]
        self._last_batch = self._next_batch()
        
    def _init_vocab(self):
        '''Must be implemented in subclasses and create the following:
                self._vocab_dict (text keys and ID values)  
                self._reverse_vocab_dict (ID keys and text values)
                self.vocab_size
            Consider also what you will do with self._vocab_size_limit'''
        raise NotImplementedError
    def _init_token_sequence(self):
        '''Must be implemented in subclasses and create the following:
                self._token_seq = list of token id's in original input order (so duplicate is ok)
                self._token_seq_size = the total # of tokens in the input stream
            Consider also what you will do with self._vocab_size_limit'''
        raise NotImplementedError

    def token_2_id(self, token):
        return self._vocab_dict[token]
    def id_2_token(self, token_id):
        return self._reverse_vocab_dict[token_id]
    def onehot_2_id(self, one_hot):
        """Turn a 1-hot encoding or a probability distribution over the possible
        characters back into its (most likely) ID representation.
        This will always return the same result for identical inputs -- it does
        not sample across the probability distribution.
        This used to be character()"""
        return [c for c in np.argmax(one_hot, 1)]
    def id_2_onehot(self, id_list):
        '''Turn a list of ids into a list of one_hot encoded vectors'''
        identity = tf.constant(np.identity(self.vocab_size, dtype = np.float32))
        return tf.nn.embedding_lookup(identity, id_list)
    
    def softmax_2_sampled_id(self, softmax_distribution):
        """Turn a softMax probability distribution over the possible
        characters into an ID representation based on a sampling over that probability
        distribution.
        This randomly samples the distribution. So, for example, if in the
        softmax distribution 'a' is 40% likely, 'b' is 40% likely and 
        'c' is 20% likely, this will generate an 'a' 40% of the time
        it is called and a 'c' 20% of the time it is called, etc."""
        r = random.uniform(0, 1)
        s = 0
        for i in range(len(softmax_distribution)):
            s += softmax_distribution[i]
            if s >= r:
                return i
        return len(softmax_distribution) - 1
    
    def _next_batch(self):
        """Generate a single row (or unrolling) of length 'batch' from the current 
        cursor position in the token data.  It will be in the form of a row of token IDs."""
        batch = np.zeros(shape=(self._batch_size, ), dtype=np.int32)
        for b in range(self._batch_size):
            batch[b] = self._token_seq[self._cursor[b]]
            self._cursor[b] = (self._cursor[b] + 1) % self._token_seq_size
        return batch
  
    def next(self):
        """Generate the next array of batches from the data. The array consists of
        the last batch of the previous array, followed by num_unrollings new ones.
        (for a total of num_unrollings + 1).
        The reason the last batch from the previous array is included is because
        in the previous array, the last batch was just used as a label to the model,
        not as an input -- so we include it this next time to be used as an input.
        Note that the sequential tokens end up being read into the columns of these
        'num_unrolling" batches.  Each column is a separate part of the token input.
        """
        batches = [self._last_batch]
        for step in range(self._num_unrollings):
            batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches
    
    def batches_2_tokens(self, batches):
        """Convert a sequence of batches back into their (most likely) string
        representation."""
        # DAY
        # This mangles the real batch structure in the interest of readability, but
        # by doing so, makes your understanding of batches wrong.
        # See 'honest_batches2string' below which gives you a better
        # understanding of the batch format.

        # batches has dimensions (num_unrollings, batch_size)
        s = [''] * batches[0].shape[0]  # batches[0].shape[0] will end up being same as batch_size
        for b in batches: # there will be num_unrollings of these...
            s = [''.join(self.id_2_token(x)) for x in zip(s, b)]  # DAY __ NEEDS WORK
        return s

    def honest_batches_2_tokens(self, batches):
        import pprint
        output = []
        for b_index, b in enumerate(batches):  # there will be 'num_unrollings' of these
            output.append(list())
            for token_id_index, token_id in enumerate(b):  # there will be 'batch_size' of these
                output[b_index].append(self.id_2_token(token_id))
        return pprint.pformat(output)

class SingleCharacterGenerator(SequenceGenerator):
    def _init_vocab(self):
        '''Must be implemented in subclasses and create the following:
                self._reverse_vocab_dict (ID keys and text values)
                self.vocab_size
            Consider also what you will do with self._vocab_size_limit'''
        self.vocab_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
        self._vocab_dict = dict()
        self._vocab_dict[' '] = 0
        for i, v in enumerate(list(string.ascii_lowercase[:self.vocab_size - 1])):
            self._vocab_dict[v] = i+1
        self._reverse_vocab_dict = dict(zip(self._vocab_dict.values(), self._vocab_dict.keys()))
    def _init_token_sequence(self):
        '''Must be implemented in subclasses and create the following:
                self._token_seq = list of token id's in original input order (so duplicate is ok)
                self._token_seq_size = the total # of tokens in the input stream
            Consider also what you will do with self._vocab_size_limit'''
        first_letter = ord(string.ascii_lowercase[0])
        self._token_seq = list()
        for char in self._text.lower():
            if char in string.ascii_lowercase:
                self._token_seq.append(self.token_2_id(char))
            elif char == ' ':
                self._token_seq.append(self.token_2_id(' '))
            else:
                pass # don't enter unknown characters (DAY should we have an UNK ID?)
        self._token_seq_size = len(self._token_seq)

class BigramGenerator(SequenceGenerator):
    def _init_vocab(self):
        '''Must be implemented in subclasses and create the following:
                self._vocab_dict (text keys and ID values)  
                self._reverse_vocab_dict (ID keys and text values)
                self.vocab_size
            Consider also what you will do with self._vocab_size_limit'''
        self._vocab_dict = dict()
        self._token_seq = list()
        id_index = 0
        for i in range(0, len(self._text)-2, 2):
            bigram = self._text[i].lower() + self._text[i+1].lower()
            if bigram not in self._vocab_dict:
                self._vocab_dict[bigram] = id_index
                id_index += 1
            self._token_seq.append(self.token_2_id(bigram)) # dup ok -- this is just input stream as token ids
        self.vocab_size = len(self._vocab_dict)
        self._reverse_vocab_dict = dict(zip(self._vocab_dict.values(), self._vocab_dict.keys()))
        self._token_seq_size = len(self._token_seq)
    def _init_token_sequence(self):
        '''Must be implemented in subclasses and create the following:
                self._token_seq = list of token id's in original input order (so duplicate is ok)
                self._token_seq_size = the total # of tokens in the input stream
            Consider also what you will do with self._vocab_size_limit'''
        pass # I did this work in _init_vocab() to use just one loop
    
class WordGenerator(SequenceGenerator):
    def _init_vocab(self):
        '''Must be implemented in subclasses and create the following:
                self._vocab_dict (text keys and ID values)  
                self._reverse_vocab_dict (ID keys and text values)
                self.vocab_size
            Consider also what you will do with self._vocab_size_limit'''
        words = self._text.lower().split()
        count = [['UNK', -1]]
        if self._vocab_size_limit is not None:
            count.extend(collections.Counter(words).most_common(self._vocab_size_limit - 1))
        else:
            count.extend(collections.Counter(words).most_common())
        self._vocab_dict = dict()
        id_index = 1
        for word, _ in count:
            self._vocab_dict[word] = id_index
            id_index += 1
        self._vocab_dict['UNK'] = 0 # force this id value for convenience
        self._token_seq = list()
        unk_count = 0
        for word in words:
            if word in self._vocab_dict:
                index = self._vocab_dict[word]
            else:
                index = 0  # self._vocab_dict['UNK']
                unk_count = unk_count + 1
            self._token_seq.append(index)
        count[0][1] = unk_count
        self.vocab_size = len(self._vocab_dict)
        self._reverse_vocab_dict = dict(zip(self._vocab_dict.values(), self._vocab_dict.keys()))
        self._token_seq_size = len(self._token_seq)
        #print('count[:1000]: %s' % count[:1000])
        del words
        del count
        # if called with build_dataset('if it is to be it is up to me'.split())
        # we get the following:
        #>>> count               # count of the '_vocab_size_limit' most common words
        #[['UNK', 3], ('is', 2), ('it', 2), ('to', 2), ('me', 1)]
        #>>> self._token_seq                # the original text as indices into 'count'
        #[0, 2, 1, 3, 0, 2, 1, 0, 3, 4]
        #>>> self._vocab_dict          # value refers to index in count
        #{'me': 4, 'to': 3, 'is': 1, 'UNK': 0, 'it': 2}
        #>>> self._reverse_vocab_dict
        #{0: 'UNK', 1: 'is', 2: 'it', 3: 'to', 4: 'me'}

        # OLD method that did not have vocab_limit
        #self._vocab_dict = dict()
        #self._token_seq = list()
        #for i, word in enumerate(self._text.lower().split()):
        #    if word not in self._vocab_dict:
        #        self._vocab_dict[word] = i
        #    self._token_seq.append(self.token_2_id(word)) # dup ok -- this is just input stream as token ids
        #self.vocab_size = len(self._vocab_dict)
        #self._reverse_vocab_dict = dict(zip(self._vocab_dict.values(), self._vocab_dict.keys()))
        #self._token_seq_size = len(self._token_seq)
    def _init_token_sequence(self):
        '''Must be implemented in subclasses and create the following:
                self._token_seq = list of token id's in original input order (so duplicate is ok)
                self._token_seq_size = the total # of tokens in the input stream
                '''
        pass # did this as part of _init_vocab()

class SingleCharWordAtTimeGenerator(SequenceGenerator):
    '''A bespoke generator for Problem 3 which is training a model to spit out mirrored words.
    For example "Now is the time" should output "woN si eht emit".
    For this, I want to feed in just one character at a time, but need this generator
    to be a bit sophisticated about how to generate the desired labels.  In order to make
    it work, I need to make sure I feed in an integral number of words (as opposed to a
    fractional number) and I need to have the algorithm use word boundaries to generate the
    right label.'''
    def _init_vocab(self):
        '''Must be implemented in subclasses and create the following:
                self._reverse_vocab_dict (ID keys and text values)
                self.vocab_size
            Consider also what you will do with self._vocab_size_limit
            
            Changed to add a padding character to the vocabulary
            '''
        self.vocab_size = len(string.ascii_lowercase) + 2 # [a-z] + ' ' + '#'
        self._vocab_dict = dict()
        self._vocab_dict[' '] = 0
        self._vocab_dict['#'] = 1 # this is the padding character
        for i, v in enumerate(list(string.ascii_lowercase[:self.vocab_size - 2])):
            self._vocab_dict[v] = i+2
        self._reverse_vocab_dict = dict(zip(self._vocab_dict.values(), self._vocab_dict.keys()))
    def _init_token_sequence(self):
        '''Must be implemented in subclasses and create the following:
                self._token_seq = list of token id's in original input order (so duplicate is ok)
                self._token_seq_size = the total # of tokens in the input stream
            Consider also what you will do with self._vocab_size_limit'''
        first_letter = ord(string.ascii_lowercase[0])
        self._token_seq = list()
        for char in self._text.lower():
            if char in string.ascii_lowercase:
                self._token_seq.append(self.token_2_id(char))
            elif char == ' ':
                self._token_seq.append(self.token_2_id(' '))
            elif char == '#':  # The padding char (although I doubt it will be in the input)
                self._token_seq.append(self.token_2_id('#'))
            else:
                pass # don't enter unknown characters (DAY should we have an UNK ID?)
        self._token_seq_size = len(self._token_seq)
    def _next_batch(self):
        """Generate a single row (or unrolling) of length 'batch' from the current 
        cursor position in the token data.  It will be in the form of a row of token IDs."""
        batch = np.zeros(shape=(self._batch_size, ), dtype=np.int32)
        for b in range(self._batch_size):
            batch[b] = self._token_seq[self._cursor[b]]
            self._cursor[b] = (self._cursor[b] + 1) % self._token_seq_size
        return batch
  
    def next(self):
        """While _next_batch() works as it is, this method (next()) needs to be
        reimplemented for this class to do the following:
        1. Choose word boundaries for the unrollings (that fit in num_unrollings)
        and then add padding to get to the num_unrolling size.  There needs to be
        at least one padding token at a minimum to serve as an end-of-input token.
        2. This will be complicated because we need to figure out the optimal
        word boundary for each of the num_batches sequences simultaneously as we
        create this batch.
        3. I don't think we need to include the last batch from the previous
        array like we did before (that was done because the last batch from the 
        previous array was only used as a label, but never as an input -- that
        won't be the case here since labels are no longer just a shift-right-one
        of the next char
        """
        batches = [self._last_batch]
        for step in range(self._num_unrollings):
            batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches



if False:
    my_text = "abcdefghijklmnopqrstuvwxyzabcdefghijklmnopqrstuvwxyz"
    #my_batches = SingleCharacterGenerator(my_text.lower(), 4, 2)
    my_batches = BigramGenerator(my_text.lower(), 4, 2)
    print(my_batches.next())
    print(my_batches.next())
    print(my_batches.next())
    print(my_batches.next())
    #my_batches = SingleCharacterGenerator(my_text.lower(), 4, 2)
    my_batches = BigramGenerator(my_text.lower(), 4, 2)
    print(my_batches.honest_batches_2_tokens(my_batches.next()))
    print(my_batches.honest_batches_2_tokens(my_batches.next()))
    print(my_batches.honest_batches_2_tokens(my_batches.next()))
    print(my_batches.honest_batches_2_tokens(my_batches.next()))
else:
    my_text = "if it is to be it is up to me"
    my_batches = WordGenerator(my_text.lower(), 4, 2)
    print(my_batches.honest_batches_2_tokens(my_batches.next()))
    print(my_batches.honest_batches_2_tokens(my_batches.next()))

[['if', 'is', 'be', 'is'], ['it', 'to', 'it', 'up'], ['is', 'be', 'is', 'to']]
[['is', 'be', 'is', 'to'], ['to', 'it', 'up', 'me'], ['be', 'is', 'to', 'if']]


In [5]:
def mirror_words(text):
    output = []
    for word in re.split('\W+', text):
        output.append(word[::-1])
    return ' '.join(output)

print(mirror_words('This is a test of mirror words.'))

sihT si a tset fo rorrim sdrow 


In [6]:
def logprob(predictions, labels):
    """Log-probability of the true labels in a predicted batch."""
    predictions[predictions < 1e-10] = 1e-10
    return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
    """Sample one element from a distribution assumed to be an array of normalized
    probabilities.
    """
    r = random.uniform(0, 1)
    s = 0
    for i in range(len(distribution)):
        s += distribution[i]
        if s >= r:
            return i
    return len(distribution) - 1

def sample(prediction, vocabulary_size):
    """Turn a (column) prediction into 1-hot encoded samples."""
    p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
    p[0, sample_distribution(prediction[0])] = 1.0  # prediction is in column format, so it must be indexed by [0]
    return p

def random_distribution(vocabulary_size):
    """Generate a random column of probabilities."""
    b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
    return b/np.sum(b, 1)[:,None]

In [7]:
batch_size=64
num_unrollings=10
num_nodes = 64
embedding_size = 10

if False:
    train_batches = SingleCharacterGenerator(train_text, batch_size, num_unrollings)
    vocabulary_size = train_batches.vocab_size
    valid_batches = SingleCharacterGenerator(valid_text, 1, 1)
else:
    train_batches = BigramGenerator(train_text, batch_size, num_unrollings)
    vocabulary_size = train_batches.vocab_size
    valid_batches = BigramGenerator(valid_text, 1, 1)


In [8]:
# Data_utils

# Copyright 2015 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

"""Utilities for downloading data from WMT, tokenizing, vocabularies."""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import gzip
import os
import re
import tarfile

from six.moves import urllib

from tensorflow.python.platform import gfile
import tensorflow as tf

# Special vocabulary symbols - we always put them at the start.
_PAD = b"_PAD"
_GO = b"_GO"
_EOS = b"_EOS"
_UNK = b"_UNK"
_START_VOCAB = [_PAD, _GO, _EOS, _UNK]

PAD_ID = 0
GO_ID = 1
EOS_ID = 2
UNK_ID = 3

# Regular expressions used to tokenize.
_WORD_SPLIT = re.compile(b"([.,!?\"':;)(])")
_DIGIT_RE = re.compile(br"\d")

# URLs for WMT data.
_WMT_ENFR_TRAIN_URL = "http://www.statmt.org/wmt10/training-giga-fren.tar"
_WMT_ENFR_DEV_URL = "http://www.statmt.org/wmt15/dev-v2.tgz"


def maybe_download(directory, filename, url):
  """Download filename from url unless it's already in directory."""
  if not os.path.exists(directory):
    print("Creating directory %s" % directory)
    os.mkdir(directory)
  filepath = os.path.join(directory, filename)
  if not os.path.exists(filepath):
    print("Downloading %s to %s" % (url, filepath))
    filepath, _ = urllib.request.urlretrieve(url, filepath)
    statinfo = os.stat(filepath)
    print("Succesfully downloaded", filename, statinfo.st_size, "bytes")
  return filepath


def gunzip_file(gz_path, new_path):
  """Unzips from gz_path into new_path."""
  print("Unpacking %s to %s" % (gz_path, new_path))
  with gzip.open(gz_path, "rb") as gz_file:
    with open(new_path, "wb") as new_file:
      for line in gz_file:
        new_file.write(line)


def get_wmt_enfr_train_set(directory):
  """Download the WMT en-fr training corpus to directory unless it's there."""
  train_path = os.path.join(directory, "giga-fren.release2.fixed")
  if not (gfile.Exists(train_path +".fr") and gfile.Exists(train_path +".en")):
    corpus_file = maybe_download(directory, "training-giga-fren.tar",
                                 _WMT_ENFR_TRAIN_URL)
    print("Extracting tar file %s" % corpus_file)
    with tarfile.open(corpus_file, "r") as corpus_tar:
      corpus_tar.extractall(directory)
    gunzip_file(train_path + ".fr.gz", train_path + ".fr")
    gunzip_file(train_path + ".en.gz", train_path + ".en")
  return train_path


def get_wmt_enfr_dev_set(directory):
  """Download the WMT en-fr training corpus to directory unless it's there."""
  dev_name = "newstest2013"
  dev_path = os.path.join(directory, dev_name)
  if not (gfile.Exists(dev_path + ".fr") and gfile.Exists(dev_path + ".en")):
    dev_file = maybe_download(directory, "dev-v2.tgz", _WMT_ENFR_DEV_URL)
    print("Extracting tgz file %s" % dev_file)
    with tarfile.open(dev_file, "r:gz") as dev_tar:
      fr_dev_file = dev_tar.getmember("dev/" + dev_name + ".fr")
      en_dev_file = dev_tar.getmember("dev/" + dev_name + ".en")
      fr_dev_file.name = dev_name + ".fr"  # Extract without "dev/" prefix.
      en_dev_file.name = dev_name + ".en"
      dev_tar.extract(fr_dev_file, directory)
      dev_tar.extract(en_dev_file, directory)
  return dev_path


def basic_tokenizer(sentence):
  """Very basic tokenizer: split the sentence into a list of tokens."""
  words = []
  for space_separated_fragment in sentence.strip().split():
    words.extend(_WORD_SPLIT.split(space_separated_fragment))
  return [w for w in words if w]


def create_vocabulary(vocabulary_path, data_path, max_vocabulary_size,
                      tokenizer=None, normalize_digits=True):
  """Create vocabulary file (if it does not exist yet) from data file.

  Data file is assumed to contain one sentence per line. Each sentence is
  tokenized and digits are normalized (if normalize_digits is set).
  Vocabulary contains the most-frequent tokens up to max_vocabulary_size.
  We write it to vocabulary_path in a one-token-per-line format, so that later
  token in the first line gets id=0, second line gets id=1, and so on.

  Args:
    vocabulary_path: path where the vocabulary will be created.
    data_path: data file that will be used to create vocabulary.
    max_vocabulary_size: limit on the size of the created vocabulary.
    tokenizer: a function to use to tokenize each data sentence;
      if None, basic_tokenizer will be used.
    normalize_digits: Boolean; if true, all digits are replaced by 0s.
  """
  if not gfile.Exists(vocabulary_path):
    print("Creating vocabulary %s from data %s" % (vocabulary_path, data_path))
    vocab = {}
    with gfile.GFile(data_path, mode="rb") as f:
      counter = 0
      for line in f:
        counter += 1
        if counter % 100000 == 0:
          print("  processing line %d" % counter)
        line = tf.compat.as_bytes(line)
        tokens = tokenizer(line) if tokenizer else basic_tokenizer(line)
        for w in tokens:
          word = _DIGIT_RE.sub(b"0", w) if normalize_digits else w
          if word in vocab:
            vocab[word] += 1
          else:
            vocab[word] = 1
      vocab_list = _START_VOCAB + sorted(vocab, key=vocab.get, reverse=True)
      if len(vocab_list) > max_vocabulary_size:
        vocab_list = vocab_list[:max_vocabulary_size]
      with gfile.GFile(vocabulary_path, mode="wb") as vocab_file:
        for w in vocab_list:
          vocab_file.write(w + b"\n")


def initialize_vocabulary(vocabulary_path):
  """Initialize vocabulary from file.

  We assume the vocabulary is stored one-item-per-line, so a file:
    dog
    cat
  will result in a vocabulary {"dog": 0, "cat": 1}, and this function will
  also return the reversed-vocabulary ["dog", "cat"].

  Args:
    vocabulary_path: path to the file containing the vocabulary.

  Returns:
    a pair: the vocabulary (a dictionary mapping string to integers), and
    the reversed vocabulary (a list, which reverses the vocabulary mapping).

  Raises:
    ValueError: if the provided vocabulary_path does not exist.
  """
  if gfile.Exists(vocabulary_path):
    rev_vocab = []
    with gfile.GFile(vocabulary_path, mode="rb") as f:
      rev_vocab.extend(f.readlines())
    rev_vocab = [line.strip() for line in rev_vocab]
    vocab = dict([(x, y) for (y, x) in enumerate(rev_vocab)])
    return vocab, rev_vocab
  else:
    raise ValueError("Vocabulary file %s not found.", vocabulary_path)


def sentence_to_token_ids(sentence, vocabulary,
                          tokenizer=None, normalize_digits=True):
  """Convert a string to list of integers representing token-ids.

  For example, a sentence "I have a dog" may become tokenized into
  ["I", "have", "a", "dog"] and with vocabulary {"I": 1, "have": 2,
  "a": 4, "dog": 7"} this function will return [1, 2, 4, 7].

  Args:
    sentence: the sentence in bytes format to convert to token-ids.
    vocabulary: a dictionary mapping tokens to integers.
    tokenizer: a function to use to tokenize each sentence;
      if None, basic_tokenizer will be used.
    normalize_digits: Boolean; if true, all digits are replaced by 0s.

  Returns:
    a list of integers, the token-ids for the sentence.
  """

  if tokenizer:
    words = tokenizer(sentence)
  else:
    words = basic_tokenizer(sentence)
  if not normalize_digits:
    return [vocabulary.get(w, UNK_ID) for w in words]
  # Normalize digits by 0 before looking words up in the vocabulary.
  return [vocabulary.get(_DIGIT_RE.sub(b"0", w), UNK_ID) for w in words]


def data_to_token_ids(data_path, target_path, vocabulary_path,
                      tokenizer=None, normalize_digits=True):
  """Tokenize data file and turn into token-ids using given vocabulary file.

  This function loads data line-by-line from data_path, calls the above
  sentence_to_token_ids, and saves the result to target_path. See comment
  for sentence_to_token_ids on the details of token-ids format.

  Args:
    data_path: path to the data file in one-sentence-per-line format.
    target_path: path where the file with token-ids will be created.
    vocabulary_path: path to the vocabulary file.
    tokenizer: a function to use to tokenize each sentence;
      if None, basic_tokenizer will be used.
    normalize_digits: Boolean; if true, all digits are replaced by 0s.
  """
  if not gfile.Exists(target_path):
    print("Tokenizing data in %s" % data_path)
    vocab, _ = initialize_vocabulary(vocabulary_path)
    with gfile.GFile(data_path, mode="rb") as data_file:
      with gfile.GFile(target_path, mode="w") as tokens_file:
        counter = 0
        for line in data_file:
          counter += 1
          if counter % 100000 == 0:
            print("  tokenizing line %d" % counter)
          token_ids = sentence_to_token_ids(line, vocab, tokenizer,
                                            normalize_digits)
          tokens_file.write(" ".join([str(tok) for tok in token_ids]) + "\n")


def prepare_wmt_data(data_dir, en_vocabulary_size, fr_vocabulary_size, tokenizer=None):
  """Get WMT data into data_dir, create vocabularies and tokenize data.

  Args:
    data_dir: directory in which the data sets will be stored.
    en_vocabulary_size: size of the English vocabulary to create and use.
    fr_vocabulary_size: size of the French vocabulary to create and use.
    tokenizer: a function to use to tokenize each data sentence;
      if None, basic_tokenizer will be used.

  Returns:
    A tuple of 6 elements:
      (1) path to the token-ids for English training data-set,
      (2) path to the token-ids for French training data-set,
      (3) path to the token-ids for English development data-set,
      (4) path to the token-ids for French development data-set,
      (5) path to the English vocabulary file,
      (6) path to the French vocabulary file.
  """
  # Get wmt data to the specified directory.
  train_path = get_wmt_enfr_train_set(data_dir)
  dev_path = get_wmt_enfr_dev_set(data_dir)

  # Create vocabularies of the appropriate sizes.
  fr_vocab_path = os.path.join(data_dir, "vocab%d.fr" % fr_vocabulary_size)
  en_vocab_path = os.path.join(data_dir, "vocab%d.en" % en_vocabulary_size)
  create_vocabulary(fr_vocab_path, train_path + ".fr", fr_vocabulary_size, tokenizer)
  create_vocabulary(en_vocab_path, train_path + ".en", en_vocabulary_size, tokenizer)

  # Create token ids for the training data.
  fr_train_ids_path = train_path + (".ids%d.fr" % fr_vocabulary_size)
  en_train_ids_path = train_path + (".ids%d.en" % en_vocabulary_size)
  data_to_token_ids(train_path + ".fr", fr_train_ids_path, fr_vocab_path, tokenizer)
  data_to_token_ids(train_path + ".en", en_train_ids_path, en_vocab_path, tokenizer)

  # Create token ids for the development data.
  fr_dev_ids_path = dev_path + (".ids%d.fr" % fr_vocabulary_size)
  en_dev_ids_path = dev_path + (".ids%d.en" % en_vocabulary_size)
  data_to_token_ids(dev_path + ".fr", fr_dev_ids_path, fr_vocab_path, tokenizer)
  data_to_token_ids(dev_path + ".en", en_dev_ids_path, en_vocab_path, tokenizer)

  return (en_train_ids_path, fr_train_ids_path,
          en_dev_ids_path, fr_dev_ids_path,
          en_vocab_path, fr_vocab_path)


In [9]:
# Seq2Seq

# Copyright 2015 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

"""Sequence-to-sequence model with an attention mechanism."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import random

import numpy as np
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

from tensorflow.models.rnn.translate import data_utils


class Seq2SeqModel(object):
  """Sequence-to-sequence model with attention and for multiple buckets.

  This class implements a multi-layer recurrent neural network as encoder,
  and an attention-based decoder. This is the same as the model described in
  this paper: http://arxiv.org/abs/1412.7449 - please look there for details,
  or into the seq2seq library for complete model implementation.
  This class also allows to use GRU cells in addition to LSTM cells, and
  sampled softmax to handle large output vocabulary size. A single-layer
  version of this model, but with bi-directional encoder, was presented in
    http://arxiv.org/abs/1409.0473
  and sampled softmax is described in Section 3 of the following paper.
    http://arxiv.org/abs/1412.2007
  """

  def __init__(self,
               source_vocab_size,
               target_vocab_size,
               buckets,
               size,
               num_layers,
               max_gradient_norm,
               batch_size,
               learning_rate,
               learning_rate_decay_factor,
               use_lstm=False,
               num_samples=512,
               forward_only=False,
               dtype=tf.float32):
    """Create the model.

    Args:
      source_vocab_size: size of the source vocabulary.
      target_vocab_size: size of the target vocabulary.
      buckets: a list of pairs (I, O), where I specifies maximum input length
        that will be processed in that bucket, and O specifies maximum output
        length. Training instances that have inputs longer than I or outputs
        longer than O will be pushed to the next bucket and padded accordingly.
        We assume that the list is sorted, e.g., [(2, 4), (8, 16)].
      size: number of units in each layer of the model.
      num_layers: number of layers in the model.
      max_gradient_norm: gradients will be clipped to maximally this norm.
      batch_size: the size of the batches used during training;
        the model construction is independent of batch_size, so it can be
        changed after initialization if this is convenient, e.g., for decoding.
      learning_rate: learning rate to start with.
      learning_rate_decay_factor: decay learning rate by this much when needed.
      use_lstm: if true, we use LSTM cells instead of GRU cells.
      num_samples: number of samples for sampled softmax.
      forward_only: if set, we do not construct the backward pass in the model.
      dtype: the data type to use to store internal variables.
    """
    self.source_vocab_size = source_vocab_size
    self.target_vocab_size = target_vocab_size
    self.buckets = buckets
    self.batch_size = batch_size
    self.learning_rate = tf.Variable(
        float(learning_rate), trainable=False, dtype=dtype)
    self.learning_rate_decay_op = self.learning_rate.assign(
        self.learning_rate * learning_rate_decay_factor)
    self.global_step = tf.Variable(0, trainable=False)

    # If we use sampled softmax, we need an output projection.
    output_projection = None
    softmax_loss_function = None
    # Sampled softmax only makes sense if we sample less than vocabulary size.
    if num_samples > 0 and num_samples < self.target_vocab_size:
      w_t = tf.get_variable("proj_w", [self.target_vocab_size, size], dtype=dtype)
      w = tf.transpose(w_t)
      b = tf.get_variable("proj_b", [self.target_vocab_size], dtype=dtype)
      output_projection = (w, b)

      def sampled_loss(inputs, labels):
        labels = tf.reshape(labels, [-1, 1])
        # We need to compute the sampled_softmax_loss using 32bit floats to
        # avoid numerical instabilities.
        local_w_t = tf.cast(w_t, tf.float32)
        local_b = tf.cast(b, tf.float32)
        local_inputs = tf.cast(inputs, tf.float32)
        return tf.cast(
            tf.nn.sampled_softmax_loss(local_w_t, local_b, local_inputs, labels,
                                       num_samples, self.target_vocab_size),
            dtype)
      softmax_loss_function = sampled_loss

    # Create the internal multi-layer cell for our RNN.
    single_cell = tf.nn.rnn_cell.GRUCell(size)
    if use_lstm:
      single_cell = tf.nn.rnn_cell.BasicLSTMCell(size)
    cell = single_cell
    if num_layers > 1:
      cell = tf.nn.rnn_cell.MultiRNNCell([single_cell] * num_layers)

    # The seq2seq function: we use embedding for the input and attention.
    def seq2seq_f(encoder_inputs, decoder_inputs, do_decode):
      return tf.nn.seq2seq.embedding_attention_seq2seq(
          encoder_inputs,
          decoder_inputs,
          cell,
          num_encoder_symbols=source_vocab_size,
          num_decoder_symbols=target_vocab_size,
          embedding_size=size,
          output_projection=output_projection,
          feed_previous=do_decode,
          dtype=dtype)

    # Feeds for inputs.
    self.encoder_inputs = []
    self.decoder_inputs = []
    self.target_weights = []
    for i in xrange(buckets[-1][0]):  # Last bucket is the biggest one.
      self.encoder_inputs.append(tf.placeholder(tf.int32, shape=[None],
                                                name="encoder{0}".format(i)))
    for i in xrange(buckets[-1][1] + 1):
      self.decoder_inputs.append(tf.placeholder(tf.int32, shape=[None],
                                                name="decoder{0}".format(i)))
      self.target_weights.append(tf.placeholder(dtype, shape=[None],
                                                name="weight{0}".format(i)))

    # Our targets are decoder inputs shifted by one.
    targets = [self.decoder_inputs[i + 1]
               for i in xrange(len(self.decoder_inputs) - 1)]

    # Training outputs and losses.
    if forward_only:
      self.outputs, self.losses = tf.nn.seq2seq.model_with_buckets(
          self.encoder_inputs, self.decoder_inputs, targets,
          self.target_weights, buckets, lambda x, y: seq2seq_f(x, y, True),
          softmax_loss_function=softmax_loss_function)
      # If we use output projection, we need to project outputs for decoding.
      if output_projection is not None:
        for b in xrange(len(buckets)):
          self.outputs[b] = [
              tf.matmul(output, output_projection[0]) + output_projection[1]
              for output in self.outputs[b]
          ]
    else:
      self.outputs, self.losses = tf.nn.seq2seq.model_with_buckets(
          self.encoder_inputs, self.decoder_inputs, targets,
          self.target_weights, buckets,
          lambda x, y: seq2seq_f(x, y, False),
          softmax_loss_function=softmax_loss_function)

    # Gradients and SGD update operation for training the model.
    params = tf.trainable_variables()
    if not forward_only:
      self.gradient_norms = []
      self.updates = []
      opt = tf.train.GradientDescentOptimizer(self.learning_rate)
      for b in xrange(len(buckets)):
        gradients = tf.gradients(self.losses[b], params)
        clipped_gradients, norm = tf.clip_by_global_norm(gradients,
                                                         max_gradient_norm)
        self.gradient_norms.append(norm)
        self.updates.append(opt.apply_gradients(
            zip(clipped_gradients, params), global_step=self.global_step))

    self.saver = tf.train.Saver(tf.all_variables())

  def step(self, session, encoder_inputs, decoder_inputs, target_weights,
           bucket_id, forward_only):
    """Run a step of the model feeding the given inputs.

    Args:
      session: tensorflow session to use.
      encoder_inputs: list of numpy int vectors to feed as encoder inputs.
      decoder_inputs: list of numpy int vectors to feed as decoder inputs.
      target_weights: list of numpy float vectors to feed as target weights.
      bucket_id: which bucket of the model to use.
      forward_only: whether to do the backward step or only forward.

    Returns:
      A triple consisting of gradient norm (or None if we did not do backward),
      average perplexity, and the outputs.

    Raises:
      ValueError: if length of encoder_inputs, decoder_inputs, or
        target_weights disagrees with bucket size for the specified bucket_id.
    """
    # Check if the sizes match.
    encoder_size, decoder_size = self.buckets[bucket_id]
    if len(encoder_inputs) != encoder_size:
      raise ValueError("Encoder length must be equal to the one in bucket,"
                       " %d != %d." % (len(encoder_inputs), encoder_size))
    if len(decoder_inputs) != decoder_size:
      raise ValueError("Decoder length must be equal to the one in bucket,"
                       " %d != %d." % (len(decoder_inputs), decoder_size))
    if len(target_weights) != decoder_size:
      raise ValueError("Weights length must be equal to the one in bucket,"
                       " %d != %d." % (len(target_weights), decoder_size))

    # Input feed: encoder inputs, decoder inputs, target_weights, as provided.
    input_feed = {}
    for l in xrange(encoder_size):
      input_feed[self.encoder_inputs[l].name] = encoder_inputs[l]
    for l in xrange(decoder_size):
      input_feed[self.decoder_inputs[l].name] = decoder_inputs[l]
      input_feed[self.target_weights[l].name] = target_weights[l]

    # Since our targets are decoder inputs shifted by one, we need one more.
    last_target = self.decoder_inputs[decoder_size].name
    input_feed[last_target] = np.zeros([self.batch_size], dtype=np.int32)

    # Output feed: depends on whether we do a backward step or not.
    if not forward_only:
      output_feed = [self.updates[bucket_id],  # Update Op that does SGD.
                     self.gradient_norms[bucket_id],  # Gradient norm.
                     self.losses[bucket_id]]  # Loss for this batch.
    else:
      output_feed = [self.losses[bucket_id]]  # Loss for this batch.
      for l in xrange(decoder_size):  # Output logits.
        output_feed.append(self.outputs[bucket_id][l])

    outputs = session.run(output_feed, input_feed)
    if not forward_only:
      return outputs[1], outputs[2], None  # Gradient norm, loss, no outputs.
    else:
      return None, outputs[0], outputs[1:]  # No gradient norm, loss, outputs.

  def get_batch(self, data, bucket_id):
    """Get a random batch of data from the specified bucket, prepare for step.

    To feed data in step(..) it must be a list of batch-major vectors, while
    data here contains single length-major cases. So the main logic of this
    function is to re-index data cases to be in the proper format for feeding.

    Args:
      data: a tuple of size len(self.buckets) in which each element contains
        lists of pairs of input and output data that we use to create a batch.
      bucket_id: integer, which bucket to get the batch for.

    Returns:
      The triple (encoder_inputs, decoder_inputs, target_weights) for
      the constructed batch that has the proper format to call step(...) later.
    """
    encoder_size, decoder_size = self.buckets[bucket_id]
    encoder_inputs, decoder_inputs = [], []

    # Get a random batch of encoder and decoder inputs from data,
    # pad them if needed, reverse encoder inputs and add GO to decoder.
    for _ in xrange(self.batch_size):
      encoder_input, decoder_input = random.choice(data[bucket_id])

      # Encoder inputs are padded and then reversed.
      encoder_pad = [data_utils.PAD_ID] * (encoder_size - len(encoder_input))
      encoder_inputs.append(list(reversed(encoder_input + encoder_pad)))

      # Decoder inputs get an extra "GO" symbol, and are padded then.
      decoder_pad_size = decoder_size - len(decoder_input) - 1
      decoder_inputs.append([data_utils.GO_ID] + decoder_input +
                            [data_utils.PAD_ID] * decoder_pad_size)

    # Now we create batch-major vectors from the data selected above.
    batch_encoder_inputs, batch_decoder_inputs, batch_weights = [], [], []

    # Batch encoder inputs are just re-indexed encoder_inputs.
    for length_idx in xrange(encoder_size):
      batch_encoder_inputs.append(
          np.array([encoder_inputs[batch_idx][length_idx]
                    for batch_idx in xrange(self.batch_size)], dtype=np.int32))

    # Batch decoder inputs are re-indexed decoder_inputs, we create weights.
    for length_idx in xrange(decoder_size):
      batch_decoder_inputs.append(
          np.array([decoder_inputs[batch_idx][length_idx]
                    for batch_idx in xrange(self.batch_size)], dtype=np.int32))

      # Create target_weights to be 0 for targets that are padding.
      batch_weight = np.ones(self.batch_size, dtype=np.float32)
      for batch_idx in xrange(self.batch_size):
        # We set weight to 0 if the corresponding target is a PAD symbol.
        # The corresponding target is decoder_input shifted by 1 forward.
        if length_idx < decoder_size - 1:
          target = decoder_inputs[batch_idx][length_idx + 1]
        if length_idx == decoder_size - 1 or target == data_utils.PAD_ID:
          batch_weight[batch_idx] = 0.0
      batch_weights.append(batch_weight)
    return batch_encoder_inputs, batch_decoder_inputs, batch_weights


In [ ]:
# Translate 

# Copyright 2015 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

"""Binary for training translation models and decoding from them.

Running this program without --decode will download the WMT corpus into
the directory specified as --data_dir and tokenize it in a very basic way,
and then start training a model saving checkpoints to --train_dir.

Running with --decode starts an interactive loop so you can see how
the current checkpoint translates English sentences into French.

See the following papers for more information on neural translation models.
 * http://arxiv.org/abs/1409.3215
 * http://arxiv.org/abs/1409.0473
 * http://arxiv.org/abs/1412.2007
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import math
import os
import random
import sys
import time

import numpy as np
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

from tensorflow.models.rnn.translate import data_utils
from tensorflow.models.rnn.translate import seq2seq_model


tf.app.flags.DEFINE_float("learning_rate", 0.5, "Learning rate.")
tf.app.flags.DEFINE_float("learning_rate_decay_factor", 0.99,
                          "Learning rate decays by this much.")
tf.app.flags.DEFINE_float("max_gradient_norm", 5.0,
                          "Clip gradients to this norm.")
tf.app.flags.DEFINE_integer("batch_size", 64,
                            "Batch size to use during training.")
tf.app.flags.DEFINE_integer("size", 1024, "Size of each model layer.")
tf.app.flags.DEFINE_integer("num_layers", 3, "Number of layers in the model.")
tf.app.flags.DEFINE_integer("en_vocab_size", 40000, "English vocabulary size.")
tf.app.flags.DEFINE_integer("fr_vocab_size", 40000, "French vocabulary size.")
tf.app.flags.DEFINE_string("data_dir", "/tmp", "Data directory")
tf.app.flags.DEFINE_string("train_dir", "/tmp", "Training directory.")
tf.app.flags.DEFINE_integer("max_train_data_size", 0,
                            "Limit on the size of training data (0: no limit).")
tf.app.flags.DEFINE_integer("steps_per_checkpoint", 200,
                            "How many training steps to do per checkpoint.")
tf.app.flags.DEFINE_boolean("decode", False,
                            "Set to True for interactive decoding.")
tf.app.flags.DEFINE_boolean("self_test", False,
                            "Run a self-test if this is set to True.")
tf.app.flags.DEFINE_boolean("use_fp16", False,
                            "Train using fp16 instead of fp32.")

FLAGS = tf.app.flags.FLAGS

# We use a number of buckets and pad to the closest one for efficiency.
# See seq2seq_model.Seq2SeqModel for details of how they work.
_buckets = [(5, 10), (10, 15), (20, 25), (40, 50)]


def read_data(source_path, target_path, max_size=None):
  """Read data from source and target files and put into buckets.

  Args:
    source_path: path to the files with token-ids for the source language.
    target_path: path to the file with token-ids for the target language;
      it must be aligned with the source file: n-th line contains the desired
      output for n-th line from the source_path.
    max_size: maximum number of lines to read, all other will be ignored;
      if 0 or None, data files will be read completely (no limit).

  Returns:
    data_set: a list of length len(_buckets); data_set[n] contains a list of
      (source, target) pairs read from the provided data files that fit
      into the n-th bucket, i.e., such that len(source) < _buckets[n][0] and
      len(target) < _buckets[n][1]; source and target are lists of token-ids.
  """
  data_set = [[] for _ in _buckets]
  with tf.gfile.GFile(source_path, mode="r") as source_file:
    with tf.gfile.GFile(target_path, mode="r") as target_file:
      source, target = source_file.readline(), target_file.readline()
      counter = 0
      while source and target and (not max_size or counter < max_size):
        counter += 1
        if counter % 100000 == 0:
          print("  reading data line %d" % counter)
          sys.stdout.flush()
        source_ids = [int(x) for x in source.split()]
        target_ids = [int(x) for x in target.split()]
        target_ids.append(data_utils.EOS_ID)
        for bucket_id, (source_size, target_size) in enumerate(_buckets):
          if len(source_ids) < source_size and len(target_ids) < target_size:
            data_set[bucket_id].append([source_ids, target_ids])
            break
        source, target = source_file.readline(), target_file.readline()
  return data_set


def create_model(session, forward_only):
  """Create translation model and initialize or load parameters in session."""
  dtype = tf.float16 if FLAGS.use_fp16 else tf.float32
  model = seq2seq_model.Seq2SeqModel(
      FLAGS.en_vocab_size,
      FLAGS.fr_vocab_size,
      _buckets,
      FLAGS.size,
      FLAGS.num_layers,
      FLAGS.max_gradient_norm,
      FLAGS.batch_size,
      FLAGS.learning_rate,
      FLAGS.learning_rate_decay_factor,
      forward_only=forward_only,
      dtype=dtype)
  ckpt = tf.train.get_checkpoint_state(FLAGS.train_dir)
  if ckpt and tf.gfile.Exists(ckpt.model_checkpoint_path):
    print("Reading model parameters from %s" % ckpt.model_checkpoint_path)
    model.saver.restore(session, ckpt.model_checkpoint_path)
  else:
    print("Created model with fresh parameters.")
    session.run(tf.initialize_all_variables())
  return model


def train():
  """Train a en->fr translation model using WMT data."""
  # Prepare WMT data.
  print("Preparing WMT data in %s" % FLAGS.data_dir)
  en_train, fr_train, en_dev, fr_dev, _, _ = data_utils.prepare_wmt_data(
      FLAGS.data_dir, FLAGS.en_vocab_size, FLAGS.fr_vocab_size)

  with tf.Session() as sess:
    # Create model.
    print("Creating %d layers of %d units." % (FLAGS.num_layers, FLAGS.size))
    model = create_model(sess, False)

    # Read data into buckets and compute their sizes.
    print ("Reading development and training data (limit: %d)."
           % FLAGS.max_train_data_size)
    dev_set = read_data(en_dev, fr_dev)
    train_set = read_data(en_train, fr_train, FLAGS.max_train_data_size)
    train_bucket_sizes = [len(train_set[b]) for b in xrange(len(_buckets))]
    train_total_size = float(sum(train_bucket_sizes))

    # A bucket scale is a list of increasing numbers from 0 to 1 that we'll use
    # to select a bucket. Length of [scale[i], scale[i+1]] is proportional to
    # the size if i-th training bucket, as used later.
    train_buckets_scale = [sum(train_bucket_sizes[:i + 1]) / train_total_size
                           for i in xrange(len(train_bucket_sizes))]

    # This is the training loop.
    step_time, loss = 0.0, 0.0
    current_step = 0
    previous_losses = []
    while True:
      # Choose a bucket according to data distribution. We pick a random number
      # in [0, 1] and use the corresponding interval in train_buckets_scale.
      random_number_01 = np.random.random_sample()
      bucket_id = min([i for i in xrange(len(train_buckets_scale))
                       if train_buckets_scale[i] > random_number_01])

      # Get a batch and make a step.
      start_time = time.time()
      encoder_inputs, decoder_inputs, target_weights = model.get_batch(
          train_set, bucket_id)
      _, step_loss, _ = model.step(sess, encoder_inputs, decoder_inputs,
                                   target_weights, bucket_id, False)
      step_time += (time.time() - start_time) / FLAGS.steps_per_checkpoint
      loss += step_loss / FLAGS.steps_per_checkpoint
      current_step += 1

      # Once in a while, we save checkpoint, print statistics, and run evals.
      if current_step % FLAGS.steps_per_checkpoint == 0:
        # Print statistics for the previous epoch.
        perplexity = math.exp(float(loss)) if loss < 300 else float("inf")
        print ("global step %d learning rate %.4f step-time %.2f perplexity "
               "%.2f" % (model.global_step.eval(), model.learning_rate.eval(),
                         step_time, perplexity))
        # Decrease learning rate if no improvement was seen over last 3 times.
        if len(previous_losses) > 2 and loss > max(previous_losses[-3:]):
          sess.run(model.learning_rate_decay_op)
        previous_losses.append(loss)
        # Save checkpoint and zero timer and loss.
        checkpoint_path = os.path.join(FLAGS.train_dir, "translate.ckpt")
        model.saver.save(sess, checkpoint_path, global_step=model.global_step)
        step_time, loss = 0.0, 0.0
        # Run evals on development set and print their perplexity.
        for bucket_id in xrange(len(_buckets)):
          if len(dev_set[bucket_id]) == 0:
            print("  eval: empty bucket %d" % (bucket_id))
            continue
          encoder_inputs, decoder_inputs, target_weights = model.get_batch(
              dev_set, bucket_id)
          _, eval_loss, _ = model.step(sess, encoder_inputs, decoder_inputs,
                                       target_weights, bucket_id, True)
          eval_ppx = math.exp(float(eval_loss)) if eval_loss < 300 else float(
              "inf")
          print("  eval: bucket %d perplexity %.2f" % (bucket_id, eval_ppx))
        sys.stdout.flush()


def decode():
  with tf.Session() as sess:
    # Create model and load parameters.
    model = create_model(sess, True)
    model.batch_size = 1  # We decode one sentence at a time.

    # Load vocabularies.
    en_vocab_path = os.path.join(FLAGS.data_dir,
                                 "vocab%d.en" % FLAGS.en_vocab_size)
    fr_vocab_path = os.path.join(FLAGS.data_dir,
                                 "vocab%d.fr" % FLAGS.fr_vocab_size)
    en_vocab, _ = data_utils.initialize_vocabulary(en_vocab_path)
    _, rev_fr_vocab = data_utils.initialize_vocabulary(fr_vocab_path)

    # Decode from standard input.
    sys.stdout.write("> ")
    sys.stdout.flush()
    sentence = sys.stdin.readline()
    while sentence:
      # Get token-ids for the input sentence.
      token_ids = data_utils.sentence_to_token_ids(tf.compat.as_bytes(sentence), en_vocab)
      # Which bucket does it belong to?
      bucket_id = min([b for b in xrange(len(_buckets))
                       if _buckets[b][0] > len(token_ids)])
      # Get a 1-element batch to feed the sentence to the model.
      encoder_inputs, decoder_inputs, target_weights = model.get_batch(
          {bucket_id: [(token_ids, [])]}, bucket_id)
      # Get output logits for the sentence.
      _, _, output_logits = model.step(sess, encoder_inputs, decoder_inputs,
                                       target_weights, bucket_id, True)
      # This is a greedy decoder - outputs are just argmaxes of output_logits.
      outputs = [int(np.argmax(logit, axis=1)) for logit in output_logits]
      # If there is an EOS symbol in outputs, cut them at that point.
      if data_utils.EOS_ID in outputs:
        outputs = outputs[:outputs.index(data_utils.EOS_ID)]
      # Print out French sentence corresponding to outputs.
      print(" ".join([tf.compat.as_str(rev_fr_vocab[output]) for output in outputs]))
      print("> ", end="")
      sys.stdout.flush()
      sentence = sys.stdin.readline()


def self_test():
  """Test the translation model."""
  with tf.Session() as sess:
    print("Self-test for neural translation model.")
    # Create model with vocabularies of 10, 2 small buckets, 2 layers of 32.
    model = seq2seq_model.Seq2SeqModel(10, 10, [(3, 3), (6, 6)], 32, 2,
                                       5.0, 32, 0.3, 0.99, num_samples=8)
    sess.run(tf.initialize_all_variables())

    # Fake data set for both the (3, 3) and (6, 6) bucket.
    data_set = ([([1, 1], [2, 2]), ([3, 3], [4]), ([5], [6])],
                [([1, 1, 1, 1, 1], [2, 2, 2, 2, 2]), ([3, 3, 3], [5, 6])])
    for _ in xrange(5):  # Train the fake model for 5 steps.
      bucket_id = random.choice([0, 1])
      encoder_inputs, decoder_inputs, target_weights = model.get_batch(
          data_set, bucket_id)
      model.step(sess, encoder_inputs, decoder_inputs, target_weights,
                 bucket_id, False)


def main(_):
  if FLAGS.self_test:
    self_test()
  elif FLAGS.decode:
    decode()
  else:
    train()

if __name__ == "__main__":
  tf.app.run()


Preparing WMT data in /tmp
